<a href="https://colab.research.google.com/github/bcsanup-github/portfolio/blob/main/NextWordPredictionUsing_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Pride and Prejudice.txt to Pride and Prejudice.txt


In [8]:
file = open("Pride and Prejudice.txt", "r", encoding = "utf8")
lines = []
for i in file:
  lines.append(i)

data = ""
for i in lines:
  data = ''.join(lines)

  data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('"', '')
  data = data.split()
  data = ''.join(data)
  data[:500]

In [9]:
len(data)

607116

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('token1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[652, 338, 339, 340, 86, 87, 341, 342, 653, 654, 655, 656, 657, 658, 659]

In [12]:
len(sequence_data)

21316

In [13]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

17378


In [15]:
sequences = []

for i in range(3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("The length of sequence are: ", len(sequences))
sequences = np.array(sequences)
print(sequences[:10])

The length of sequence are:  21313
[[652 338 339 340]
 [338 339 340  86]
 [339 340  86  87]
 [340  86  87 341]
 [ 86  87 341 342]
 [ 87 341 342 653]
 [341 342 653 654]
 [342 653 654 655]
 [653 654 655 656]
 [654 655 656 657]]


In [16]:
x = []
y = []

for i in sequences:
  x.append(i[0:3])
  y.append(i[3])

x = np.array(x)
y = np.array(y)

In [17]:
print("Data: ", x[:10])
print("Response: ", y[:10])

Data:  [[652 338 339]
 [338 339 340]
 [339 340  86]
 [340  86  87]
 [ 86  87 341]
 [ 87 341 342]
 [341 342 653]
 [342 653 654]
 [653 654 655]
 [654 655 656]]
Response:  [340  86  87 341 342 653 654 655 656 657]


In [18]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_shape=(3,)))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 3, 10)               │         173,780 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 3, 1000)             │       4,044,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 1000)                │       8,004,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 17378)               │      17,395,378 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,618,158 (116.80 MB)

 Trainable params: 30,618,158 (116.80 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(x, y, epochs=10, batch_size=64, callbacks=[checkpoint])

Epoch 1/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2254
Epoch 1: loss improved from inf to 0.31900, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 23s 61ms/step - loss: 0.2260
Epoch 2/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1392
Epoch 2: loss improved from 0.31900 to 0.15795, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 0.1393
Epoch 3/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0989
Epoch 3: loss improved from 0.15795 to 0.13595, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - loss: 0.0992
Epoch 4/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1280
Epoch 4: loss did not improve from 0.13595
334/334 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - loss: 0.1281
Epoch 5/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1419
Epoch 5: loss did not improve from 0.13595
334/334 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - loss: 0.1421
Epoch 6/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1122
Epoch 6: loss did not improve from 0.13595
334/334 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - loss: 0.1124
Epoch 7/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1180
Epoch 7: loss improved from 0.13595 to 0.12631, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 31s 62ms/step - loss: 0.1180
Epoch 8/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0925
Epoch 8: loss improved from 0.12631 to 0.10582, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 41s 62ms/step - loss: 0.0926
Epoch 9/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0827
Epoch 9: loss improved from 0.10582 to 0.10071, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 34s 42ms/step - loss: 0.0828
Epoch 10/10
333/334 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0630
Epoch 10: loss improved from 0.10071 to 0.08252, saving model to next_words.h5


334/334 ━━━━━━━━━━━━━━━━━━━━ 28s 63ms/step - loss: 0.0631


In [29]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

model = load_model('next_words.h5')
tokenizer = pickle.load(open('token1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array (sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")

  if text == "0":
    print("Execution completed...")
    break

  else:
     try:
        text = text.split(" ")
        text = text[-3:]
        print(text)
        Predict_Next_Words(model, tokenizer, text)

     except Exception as e:
        print("Error occured: ", e)
        continue

Enter your line: The Project Gutenberg
['The', 'Project', 'Gutenberg']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
mr
Enter your line: one great difference
['one', 'great', 'difference']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
wereitnotfortheinferiorityofherconnections
Enter your line: But that is
['But', 'that', 'is']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
iquitedetesttheman
